In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
import os 
from langchain_openai import ChatOpenAI

os.environ["GROQ_API_KEY"] = "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"


In [4]:
loader = PyPDFLoader(r"C:\Users\Gautham\Documents\Gautham_resume.pdf")
pages = loader.load_and_split()

In [7]:
resume_content = ''
for page in pages:
    resume_content += page.page_content
print(resume_content)

GAUTHAM BALRAJ
+91 80729 27699 ⋄Coimbatore, TN
gauthambalraj07@gmail.com ⋄linkedin.com/gautham-balraj ⋄github
PROFILE
Data science student and Generative AI enthusiast with skills in machine learning, AI, and data analytics, seeking
an internship to apply my knowledge to real-world projects and drive innovation in the field.
EDUCATION
Master of Science (Integrated) , AMRITA VISHWA VIDHYAPEETHAM 2020 - Present
Data Science
Current CGPA: 8.04
Secondary Education , Srinivasa Vidhyalaya March 2020
Percentage: 82.5
Higher Secondary Education , Srinivasa Vidhyalaya April 2018
Percentage: 92
SKILLS
Technical Skills - Generative AI, Machine Learning, Deep Learning, Data Analytics, Big Data (Hadoop), NLP
Tools & Libraries - Tensorflow, Pytorch, Langchain, Sckit-learn, PowerBI, Jupyter Lab, AWS
Soft Skills - Communication, Problem-solving, Collaboration, Adaptability, Attention to detail
Communication - Tamil (Native), English (Professional)
EXPERIENCE
Data science Intern Dec 2022 - Jan 2023
Iam

In [14]:
SystemMessage = """Act as either a recruiter. Based on the provided unstructrued content of the resume , your role to format and provide the structured content of the resume, dont make any changes to the content of the resume.
resume content : {resume_content}"""
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SystemMessage
        ),
    ])
model = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1", # https://api.openai.com/v1 or https://api.groq.com/openai/v1 
    openai_api_key= os.getenv("GROQ_API_KEY"), 
    model_name="llama3-70b-8192",
    temperature=0
)


    #model  = ChatGroq(temperature=0, groq_api_key="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", model_name="mixtral-8x7b-32768")
   
chain = prompt | model | StrOutputParser()

In [19]:
resume_formatted = chain.invoke({"resume_content":resume_content})


In [32]:
chat_history = []
role  = "Machine Learning Engineer"
level  = "Junior"
number_of_questions = 4
system_message_2 = f"""Act as an experienced Human Resources professional specializing in {role}. Your task is to conduct an interview based on the provided resume and the specified role and level.
Resume: {resume_formatted}
Instructions:

Thoroughly review the resume and formulate {number_of_questions} relevant questions for the {level} level position.
Prioritize questions based on the candidate's experience, projects, and skills mentioned in the resume. If needed, ask a few additional questions related to the role.
Ask one question at a time and wait for the candidate's response before proceeding to the next question.
If the candidate provides an unsatisfactory or incorrect answer, give a brief response and move on to the next question.
Keep the questions concise and specific to allow for short responses from the candidate.
If the candidate's response warrants a follow-up question, ask it before moving on to the next question.
After asking all the questions, conclude the interview by saying, "Thank you for your time."

Remember to maintain a professional and objective tone throughout the interview process."""
def get_response(chat_history):
    system_message = f"""Act as a seasoned Human Resources professional specializing in {role} your responsibility is to see the resume and
                take the interview on that subject, where you will be asking only one of question for {level} level like an interviewer nothing else.
                This is the content of the reusme {resume_formatted} refer to this and ask the question
                Dont elaborate to much on the candidate's response just provide a short response and go for the next question",
                if the candidate is not able to answer the question or provided the wrong answer, just provide a short response based on it and move to next question.
                just ask only {number_of_questions} questions one by one and based on the response  given by the candidate and go for a follow up question if needed.
                Ask the questions in a way that the candidate can answer in a short response.
                ask most  of the questions based on the resume and rest of the questions based on the role,
                If there is any experience or project mentioned in the resume ask the question based on that.
                ONLY ASK ONE QUESTION AT A TIME, when you done with asking all the questions repond with "Thank you for your time" """
    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_message_2
        ),
        MessagesPlaceholder(variable_name="messages"),
    ])

   
    chain = prompt | model | StrOutputParser()
    
    return chain.invoke({
        "messages": chat_history
    })
chat_history.append(HumanMessage(content="lets start the interview"))
ai_response = get_response(chat_history)
print(ai_response)
while True:
    user_input = input("You: ")
    chat_history.append(HumanMessage(content=user_input))
    ai_response = get_response(chat_history)
    print(ai_response)
    chat_history.append(AIMessage(content=ai_response))
    if len(chat_history) == number_of_questions*2:
        break


Let's begin the interview. Here's my first question:

Can you elaborate on your experience with Generative AI, specifically in your projects, such as Corrective RAG and Auto Podcast Generator? How did you utilize LangChain, Gemini, and Groq API in these projects, and what were some of the challenges you faced?

Please respond, and I'll proceed with the next question based on your answer.
Let's begin the interview. Here's my first question:

Can you walk me through your experience with Generative AI projects, specifically how you handled data quality and model coherence in your projects, such as the Corrective RAG or Auto Podcast Generator?
It seems like you're providing some context about Generative AI projects. That's great! However, I'd like to proceed with the interview. Here's my first question again:

Can you walk me through your experience with Generative AI projects, specifically how you handled data quality and model coherence in your projects, such as the Corrective RAG or Aut

BadRequestError: Error code: 400 - {'error': {'message': 'messages.8.content: user message cannot be an null or empty', 'type': 'invalid_request_error'}}

In [27]:
chat_history

[HumanMessage(content='lets start the interview'),
 HumanMessage(content='The main difference lies in their purpose: a generative model learns the joint probability distribution of the input features and labels, allowing it to generate new samples, while a discriminative model directly learns the conditional probability of the labels given the input features, focusing on classification tasks. Generative models are useful for tasks like data generation, while discriminative models excel in classification tasks, especially when labeled data is abundant.'),
 AIMessage(content="That's correct! Generative models, such as GANs and VAEs, learn the underlying distribution of the data and can generate new samples, whereas discriminative models, like logistic regression and SVM, focus on classification tasks and learn the conditional probability of the labels given the input features.\n\nHere's my next question:\n\nWhat is the purpose of regularization in machine learning, and how does it help i

In [33]:
from pyht import Client, TTSOptions, Format
import io

# Initialize PlayHT API with your credentials
client = Client("g3wZk4yPnQNsFREn0MNRNadETv02", "c6d83fa14e4f401cb64e05e2643dddc0")

def text_to_mp3(text, filename="output.mp3", voice_engine="PlayHT2.0-turbo", sample_rate=44100, speed=1):
    # Configure TTS options
    options = TTSOptions(
        voice="s3://voice-cloning-zero-shot/9fc626dc-f6df-4f47-a112-39461e8066aa/oliviaadvertisingsaad/manifest.json",
        sample_rate=sample_rate,
        format=Format.FORMAT_MP3,
        speed=speed
    )

    # Generate audio stream
    audio_stream = io.BytesIO()
    for chunk in client.tts(text=text, voice_engine=voice_engine, options=options):
        audio_stream.write(chunk)

    # Write audio stream to file
    with open(filename, "wb") as f:
        f.write(audio_stream.getvalue())

# Example usage
text = "Hey, this is Jennifer from Play."
text_to_mp3(text, "output.mp3")


In [1]:
# `pip3 install assemblyai` (macOS)
# `pip install assemblyai` (Windows)

import assemblyai as aai

aai.settings.api_key = "d0d54f4f51284c419b0dec49548b5e3a"
transcriber = aai.Transcriber()

transcript = transcriber.transcribe("E:\code_files\Banao_Intern\AI_Interview\output.mp3")
# transcript = transcriber.transcribe("./my-local-audio-file.wav")

print(transcript.text)

Hi again. It seems like we got disconnected for a moment. Let's get back on track. You were about to share an example of a project where you've used one of the tools you've listed. Could you please share that with me?


In [1]:
from faster_whisper import WhisperModel

model = WhisperModel("tiny")

segments, info = model.transcribe("E:\code_files\Banao_Intern\AI_Interview\output.mp3")
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))


Estimating duration from bitrate, this may be inaccurate


[0.00s -> 4.80s]  Welcome to our Augmented Interview. Please start by telling us a bit about yourself.


In [3]:
# pip install requests
import requests

url = "https://api.lemonfox.ai/v1/audio/transcriptions"
headers = {
  "Authorization": "buu3zOdoZWbu0K3WHUDQbTGOcteeKaEP"
}
data = {
  "file": "E:\code_files\Banao_Intern\AI_Interview\\alert_gas_leak.mp3",
  "language": "english",
  "response_format": "json"
}

# To upload a local file add the files parameter:
files = {"file": open("E:\code_files\Banao_Intern\AI_Interview\\alert_gas_leak.mp3", "rb")}
response = requests.post(url, headers=headers, files=files, data=data)
print(response.json())

{'text': 'Emergency. Gas leak detected on your premises evacuate immediately.'}


In [5]:
from openai import OpenAI

client = OpenAI(
  api_key="buu3zOdoZWbu0K3WHUDQbTGOcteeKaEP",
  base_url="https://api.lemonfox.ai/v1",
)

audio_file = open(r"E:\code_files\Banao_Intern\AI_Interview\output.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file,
  language="en",
  response_format="text"
)

In [6]:
transcript

'Welcome to our augmented interview. Please start by telling us a bit about yourself.'

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
import os 
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
import streamlit as st
from streamlit_modal import Modal
import tempfile
from pathlib import Path
from pyht import Client, TTSOptions, Format
import io
from utils import text_to_mp3, autoplay_audio
from audio_test import record_text,transcribe_audio
import time
from streamlit.components.v1 import html
from streamlit_js_eval import streamlit_js_eval
from colour_print import print_red, print_green, print_yellow, print_blue, print_magenta, print_cyan, print_white
os.environ["GROQ_API_KEY"] = "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"


## class for structured output (pydantic)

class percentage(BaseModel):
    score: int = Field(description="The percentage score for the candidate's performance in the interview.")




In [16]:

chat = """"
Interviewer: Hi, thank you for joining us today. Could you start by telling me a bit about yourself and your professional background?

Candidate: Certainly. My name is Alex Johnson,

Interviewer: Great. Can you describe a challenging project you worked on recently and how you handled it?

Candidate: can we skip thiss 
"""

In [17]:
from langchain_groq import ChatGroq

def evaluate_percentage(chat_history,role,level):
    evaluation_prompt = """
    As an experienced HR professional, review the following interview conversation history to evaluate the candidate's performance for the role of {role} at the {level} difficulty. The evaluation should consider various aspects of the interview process and be strict in assigning a percentage score based on the provided conversation history.
    Conversation History:
    {conversation_history}

    
    
    Please provide a percentage score (don't add any preamble or explanation):
    """

    chat = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    api_key= os.getenv("GROQ_API_KEY"))


    structued_llm = chat.with_structured_output(percentage)
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You're an experienced HR professional evaluating a candidate's interview performance."),
            ("human", evaluation_prompt),
        ]
    )

    chain = prompt | structued_llm 
    


    return chain.invoke({"conversation_history": chat_history, "role": role, "level": level})
result = evaluate_percentage(chat,"Machine Learning Engineer","Junior")

In [18]:
result

percentage(score=0)

In [33]:
from pydantic import BaseModel, Field
import os
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

class InterviewEvaluation(BaseModel):
    percentage: int = Field(..., description="The strict performance rating percentage based on the interview conversation history.")
    review: str = Field(..., description="A comprehensive evaluation of the candidate's performance based on the interview conversation history.")

def evaluate_interview(chat_history, role, level):
    # Evaluation prompt for detailed review
    review_prompt = """
    As an experienced HR professional, review the following conversation history of an interview 
    and provide a comprehensive evaluation of the candidate's performance. Your evaluation should include:

    1. Summary of Key Points Discussed: Highlight the main topics and points covered during the interview.
    2. Performance Rating: Assign a performance rating as a percentage based on the candidate's responses.
    3. Fit for the Role: Based solely on the conversation (and not the resume), assess whether the candidate 
       seems to be a good fit for the role.

    When evaluating, consider the following aspects:
    - Clarity of Communication: How clearly did the candidate express their thoughts and ideas?
    - Relevance of Answers: Were the candidate’s answers pertinent to the questions asked and the role applied for?
    - Depth of Knowledge: Did the candidate demonstrate a thorough understanding of the subject matter?
    - Overall Impression: What was your overall impression of the candidate's suitability for the role?

    Conversation History:
    {chat_history}

    Please provide a detailed summary, a performance rating percentage, and an assessment of the candidate's fit for the role based solely on the conversation.
    Important Note: Ensure each user response in the conversation history fully answers the question asked without beating around the bush.
    and be very strict in rating the performance of the candidate and fitness for the role.
    """

    # Evaluation prompt for percentage score
    percentage_prompt = """
    As an experienced HR professional, review the following interview conversation history to evaluate the candidate's performance for the role of {role} at the {level} difficulty. The evaluation should consider various aspects of the interview process and be strict in assigning a percentage score based on the provided conversation history.

    Conversation History:
    {chat_history}


    Note: dont provide any explanation or preamble, just provide the percentage score based on the conversation history.
    Please provide a percentage score (don't add any preamble or explanation):
    """

    # Model for detailed review
    review_model = ChatOpenAI(
        openai_api_base="https://api.groq.com/openai/v1",
        openai_api_key=os.getenv("GROQ_API_KEY"),
        model_name="llama3-70b-8192",
        temperature=0
    )

    # Model for percentage score
    percentage_model = ChatGroq(
        temperature=0,
        model="llama3-70b-8192",
        api_key=os.getenv("GROQ_API_KEY")
    )

    # Create prompt templates
    prompt_template_review = ChatPromptTemplate.from_messages(
        [
            ("system", "You're an experienced HR professional evaluating a candidate's interview performance."),
            ("human", review_prompt),
        ]
    )
    prompt_template_precentage = ChatPromptTemplate.from_messages(
        [
            ("system", "You're an experienced HR professional evaluating a candidate's interview performance."),
            ("human", percentage_prompt),
        ]
    )

    # Create chains
    review_chain = prompt_template_review | review_model | StrOutputParser()
    percentage_chain =  prompt_template_precentage | percentage_model.with_structured_output(percentage)    

    # Invoke the chains
    review_result = review_chain.invoke({"chat_history": chat_history})
    percentage_result = percentage_chain.invoke({"chat_history": chat_history, "role": role, "level": level})

    # Return results in a Pydantic object
    return InterviewEvaluation(
        percentage=percentage_result.score,
        review=review_result
    )

# Example usage

role = "Machine Learning Engineer"
level = "Junior"
result = evaluate_interview(chat, role, level)
print(result.percentage, result.review,sep='\n')


0
**Summary of Key Points Discussed:**

Unfortunately, the conversation history is quite limited, and the candidate failed to provide any meaningful information about themselves or their professional background. The candidate was asked to describe a challenging project they worked on recently and how they handled it, but instead of answering the question, they requested to skip it.

**Performance Rating:**

Based on the conversation history, I would assign a performance rating of 10% to the candidate. The candidate failed to provide any relevant information about themselves or their experience, and they demonstrated a lack of willingness to answer a critical question about their problem-solving skills.

**Fit for the Role:**

Based solely on the conversation, I would assess that the candidate is not a good fit for the role. The candidate's inability to provide basic information about themselves and their experience raises concerns about their communication skills, motivation, and willi

In [28]:
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_groq import ChatGroq
import os

# Set environment variables
os.environ["GROQ_API_KEY"] = "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"
os.environ['OPENAI_API_KEY'] = "sk-gautham-ppZK3HhAOqSXO2jIOkqnT3BlbkFJXobfeGpJZ4KAQfrbsur3"

# Define InterviewAreas model
class InterviewAreas(BaseModel):
    areas: List[str] = Field(
        description="List of 5 main areas to test in the interview",
    )

# Initialize LLMs
gpt = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
groq = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
)

def get_interview_areas(role: str, years_of_experience: str, llm = groq) -> InterviewAreas:
    system_msg = f"""
    For this given 'Role: {role} | Experience: {years_of_experience} years', provide 5 very important core technical concepts to test in an interview.
    Output the areas in JSON format: {{"areas": ["area1", "area2", "area3", "area4", "area5"]}}
    just provide the output dont add any preamble or explanation
    """
    llm_with_structure = llm.with_structured_output(InterviewAreas)
    ai_msg = llm_with_structure.invoke(system_msg)
    print(llm)
    return ai_msg.areas

# Example usage
role = "AI Engineer"
years_of_experience = "3"
llm = gpt

interview_areas = get_interview_areas(role, years_of_experience,llm=gpt)
print(interview_areas)


client=<openai.resources.chat.completions.Completions object at 0x00000212A94023E0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000212A90F1E40> model_name='gpt-3.5-turbo-0125' temperature=0.0 openai_api_key=SecretStr('**********') openai_proxy=''
['Machine Learning Algorithms', 'Deep Learning Architectures', 'Natural Language Processing', 'Computer Vision', 'Reinforcement Learning']


In [25]:
chat  = """
Interviewer: Welcome, thanks for joining us today. Can you start by telling me a bit about your experience with machine learning?

Candidate: Absolutely. I have been working in machine learning for the past five years. I started with supervised learning techniques, then moved on to unsupervised learning, and now I’m heavily involved in deep learning projects.

Interviewer: That's great to hear. Can you give an example of a project where you applied deep learning?

Candidate: Sure, one of the recent projects I worked on was developing a convolutional neural network for image classification. We used TensorFlow and Keras to build the model, and it significantly improved the accuracy compared to our previous methods.

Interviewer: Interesting. Could you explain the preprocessing steps you followed for the image data?

Candidate: Of course. We performed several preprocessing steps including image resizing, normalization, and augmentation techniques like rotation and flipping to increase the diversity of our training data.

Interviewer: That sounds comprehensive. How did you handle model evaluation and validation?

Candidate: We used a combination of techniques such as cross-validation, confusion matrix analysis, and ROC curves. Additionally, we set aside a validation set from our training data to fine-tune the model’s hyperparameters.

Interviewer: Can you describe a challenge you faced during this project and how you overcame it?

Candidate: One major challenge was dealing with overfitting. Initially, our model performed exceptionally well on the training data but poorly on the validation set. We tackled this by implementing dropout layers and using early stopping during training.

Interviewer: Very insightful. Now, can you discuss your experience with deploying machine learning models in a production environment?

Candidate: Sure. In my current role, I’ve deployed several models using Docker and Kubernetes. We often use CI/CD pipelines to automate the deployment process, ensuring that our models can be easily updated and scaled.

Interviewer: Great. Lastly, could you tell me about a time you had to collaborate with a team to complete a machine learning project?

Candidate: Collaboration is key in our projects. For the image classification project, I worked closely with data engineers to manage the data pipeline and with software developers to integrate the model into our application. Regular meetings and clear communication were crucial to our success.

Interviewer: Thank you for sharing your experiences. That’s all the questions I have for now. Do you have any questions for me?

Candidate: Not at the moment, but thank you for the opportunity to discuss my experience.

Interviewer: Thank you for your time.
"""
role = "Machine Learning Engineer"
years_of_experience = "2"


In [53]:
from pydantic import BaseModel, Field, validator
from typing import Dict
from langchain_core.prompts import ChatPromptTemplate
import json 


class AreaPercentageModel(BaseModel):
    areas: Dict[str, int] = Field(description="Areas with respective percentages")
    @classmethod
    def from_json_string(cls, json_string: str):
        # Remove newline characters and extra spaces
        clean_string = json_string.replace("\n", "").replace(" ", "")
        # Parse the cleaned string into a dictionary
        data = json.loads(clean_string)
        return cls(areas=data)
def skills_based(chat_history, role, level, areas):
    review_prompt = """\
    As an experienced HR professional, review the following interview conversation history to evaluate the candidate's performance for the role of {role} at the {level} difficulty. The evaluation should be strictly based on the provided conversation history.

    Please provide a percentage for each area in this list: {areas} based on the conversation history. The output should be in JSON format: {{"area1": percentage, "area2": percentage, ...}}. Just provide the output, don't add any preamble or explanation. Don't provide 0 percentage for any area, provide the percentage based on the conversation history.
    
    Conversation History:
    {conversation_history}
    """

    structued_llm = gpt.with_structured_output(AreaPercentageModel)
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You're an experienced HR professional evaluating a candidate's interview performance."),
            ("human", review_prompt),
        ]
    )

    chain = prompt | gpt 
    result = chain.invoke({"conversation_history": chat_history, "role": role, "level": level, "areas": areas})
    result = AreaPercentageModel.from_json_string(result.content)
    return result.areas

    



skills_based(chat,role,years_of_experience,interview_areas)


{'ExperiencewithMachineLearning': 90,
 'ProjectExamples': 85,
 'DataPreprocessingTechniques': 80,
 'ModelEvaluationandValidation': 85,
 'Problem-Solving': 85,
 'DeploymentofMLModels': 90,
 'CollaborationandTeamwork': 85}